<a href="https://colab.research.google.com/github/lagom-QB/M11/blob/master/Practice_5_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Convolutional Neural Network

# keywords: convolution, batchnorm, vgg, inception, cifar

# Goodle Drive

One of the big disadvantages of colab is the absence of persistent drive. 

We can solve this problem, by mounting google drive to colab.

In [0]:
from google.colab import drive

In [2]:
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


This command will ask you for authorization code. You can get it by following the URL printed (you will ask to allow colab to access your gdrive data). 

In [3]:
!ls /content/drive/My\ Drive/Colab\ Notebooks/DS411

ls: cannot access '/content/drive/My Drive/Colab Notebooks/DS411': No such file or directory


No your drive (and drives shared with you) can be accessed from the notebook. Data stored in your drive can be accessed from later sessions.
Beware that all changes are permanent. Do not delete data you may need later from your google drive.

# VGG16

In [0]:
from torch import nn
import torch


Convolution layer are already implemented in pytorch module library.

In [0]:
conv = nn.Conv2d(3, 3, kernel_size=5, padding=2)
conv

In [0]:
list(conv.named_parameters())

In [0]:
import torchvision
from torchvision import transforms
from torchvision import models

torchvision module has many models and helper functions, usefull for cv

In [0]:
vgg16 = models.vgg16()

In [0]:
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

In [0]:
!rm -r logs

In [0]:
%tensorboard --logdir logs

Let's create a fake input bath of 32x32 rgb images (first dimension in pytorch is always a batchsize)

In [0]:
input_batch = torch.ones((64, 3, 32, 32))

In [0]:
writer = SummaryWriter(log_dir="logs/vgg16_graph")
writer.add_graph(vgg16, input_batch)
writer.close()

In [0]:
vgg16

Most of the models can also be downloaded with weights pretrained on imagenet (it's an image dataset with 1000 classes)

In [0]:
vgg16_pretrained = models.vgg16(pretrained=True)

In [0]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
toPIL = transforms.ToPILImage()

In [0]:
toPIL(list(vgg16_pretrained.features[28].parameters())[0][67].cpu()).resize((256, 256))

# CIFAR10

In [0]:
transform = transforms.Compose(
    [transforms.ToTensor()])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4,
                                          shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:

def example(dataset, i):
    print(classes[dataset[i][1]])
    return toPIL(dataset[i][0])

In [0]:
example(train_dataset, 13).resize((256, 256))


# Augmentations

In matine learning applications, you always want to get as much diverse data as you can.

Augmentations is a way to get some "new" data for free.
There are plenty of augmentations in `torchvision` library, but `albumentations` is one of the best framework-independent augmentation library.

In [0]:
import albumentations as alb
import albumentations.augmentations.transforms as aat
import numpy as np
from PIL import Image


Let's write a simple adapter between albumentation and pytorch transformations

In [0]:
class AlbuWrapper:  # typing: ignore
    def __init__(self, atrans: alb.BasicTransform):
        self.atrans = atrans

    def __call__(self, img: Image.Image) -> Image.Image:
        return self.atrans(image=np.array(img))["image"]

In [0]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [0]:
alb_transforms = alb.Compose(
        [
            alb.OneOf([alb.IAAAdditiveGaussianNoise(), alb.GaussNoise()], p=0.2),
            alb.OneOf(
                [alb.MotionBlur(p=0.2), alb.MedianBlur(blur_limit=3, p=0.1), alb.Blur(blur_limit=3, p=0.1)], p=0.2
            ),
            alb.OneOf([alb.OpticalDistortion(p=0.3), alb.GridDistortion(p=0.1), alb.IAAPiecewiseAffine(p=0.3)], p=0.2),
            alb.OneOf([aat.CLAHE(clip_limit=2), alb.IAASharpen(), alb.IAAEmboss()], p=0.3),
            aat.HueSaturationValue(p=0.3),
            aat.HorizontalFlip(),
            aat.RGBShift(),
            aat.RandomBrightnessContrast(),
            aat.RandomGamma(),
            aat.Cutout(2, 10, 10)
        ])

In [0]:
test_transforms = transform


In [0]:
train_transforms = transforms.Compose(
[AlbuWrapper(alb_transforms), transform])

In [0]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transforms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=258,
                                          shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=258,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Let's have a look at transformed bird:

In [0]:
example(train_dataset, 13).resize((256, 256))


# Replace classifier


VGG model we loaded is designed for imagenent. Imagenet has 1000 classes and the pictures there are larger.

We can use feature extractor as-is without any changes (CNN).
But we will need a new dense classifier (we have only 10 classes).

In [0]:
classifier = nn.Sequential(nn.Linear(25088, 4096), nn.ReLU(), nn.Linear(4096, 4096), nn.ReLU(), nn.Linear(4096,10))

In [0]:
vgg16.classifier = classifier

In [0]:
writer = SummaryWriter(log_dir="logs/vgg16_graph_new_classifier")
writer.add_graph(vgg16, input_batch)
writer.close()

### What problems do you see in this model?

Dense classiffier is 49 times larger than needed...

In [0]:
vgg16.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))

Now we have 512 dimensional feature vector. Probably there is no need in 2 hidden layers, so we can simplify classifier a bit.

In [0]:
classifier = nn.Sequential(nn.Linear(512, 512), nn.ReLU(), nn.Linear(512,10))

In [0]:
vgg16.classifier = classifier

# Pretrain

We have a VGG16 network pretrained on imagenet. Even though this dataset is pretty different from CIFAR, the first few layers of feature extractor usually are pretty generic and we can reuse them.

In [0]:
vgg16_pretrained.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))

In [0]:
vgg16_pretrained.classifier = classifier

In [0]:
vgg16_pretrained

Let's **freeze** the first 19 layers of feature extractor.

In [0]:
for layer in vgg16_pretrained.features[0:19]:
  for p in layer.parameters():
    p.requires_grad = False

Now when we optimize the model, the weights of the first 19 layers will not be updated.

Once we trained models with the frozen layers it may be benefitial to "unfreeze" frozen layers and train the whole model. Usually you will use SGD optimizer with a really small learning rate for the last part.

In [0]:
for layer in vgg16_pretrained.features[0:19]:
  for p in layer.parameters():
    p.requires_grad = False

# GPU

In [0]:
import torch

In [0]:
x = torch.tensor([1,2,3])

In [0]:
y = torch.tensor([1,2,3], device=torch.device("cuda"))

In [0]:
x

In [0]:
y

In [0]:
x + y

In [0]:
x.to(torch.device("cuda")) + y

In [0]:
x + y.to(torch.device("cpu"))

In [0]:
x

Remember that your model also have weights. So, if you want to train on GPU, you need to move both, model weights, data and labels to GPU.

In [0]:
device = torch.device("cuda")
model = model.to(device)
data = data.to(device)
label = label.to(device)

# Assignment [10]

1. Update your `train` and `test` procedures, so they can accept additional argument `device` and can be used for both cpu and gpu training. [1]
2. Try taining an epoch on cifar on cpu (do not use augmentations for now) and on gpu. Compare the training speed. [1]
3. Try taining an epoch on cifar on gpu (do not use augmentations for now) with old and updated classifier part. Compare the training speed. [1]
4. Train vgg16 cifar classifier on gpu with updated classifier and no augmentations.[1]
5. Train vgg16 cifar classifier on gpu with updated classifier and augmentations.[1]
6. Train vgg16 cifar classifier initialized with pretrained weights (do not freeze layers).[2]
7. Train vgg16 cifar classifier initialized with pretrained weights with first 19 layers frozen, then unfreeze those layers and continue training with a small learning rate and SGD classifier. [3]